In [21]:
import pandas as pd
df = pd.read_csv('firefoxbuglist.csv')
df['Summary']= df['Summary'].astype(str)
ratings = list(df["Severity"])
reviews = list(df["Summary"])

In [22]:
import nltk
from nltk.corpus import opinion_lexicon
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('opinion_lexicon') #Download Opinion Dictionary
positive_wds = set(opinion_lexicon.positive())
negative_wds = set(opinion_lexicon.negative())



[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/abyte/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!


In [23]:
def score_sent(sent):
    """Returns a score btw -1 and 1"""
    sent = [e.lower() for e in sent if e.isalnum()]
    total = len(sent)
    pos = len([e for e in sent if e in positive_wds])
    neg = len([e for e in sent if e in negative_wds])
    if total > 0:
        return (pos - neg) / total
    else:
        return 0


In [24]:
def score_review(review):
    sentiment_scores = []
    sents = sent_tokenize(review)
    for sent in sents:
        wds = word_tokenize(sent)
        sent_scores = score_sent(wds)
        sentiment_scores.append(sent_scores)
    return sum(sentiment_scores) / len(sentiment_scores)

In [25]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/abyte/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
review_sentiments = [score_review(e) for e in reviews]
df = pd.DataFrame(
    {
        "rating": ratings,
        "review": reviews,
        "review dictionary based sentiment": review_sentiments,
    }
)
df.to_csv('dictionary_based_sentiment.csv')

In [27]:
def score_to_rating(value):
    if value > 0.2:
        return 2
    if value <= 0.2 and value >= -0.2:
        return 1
    else:
        return 0
df['predicted'] = df['review dictionary based sentiment'].apply(lambda x:score_to_rating(x))

In [28]:
def score_to_Target(value):
    if value >= 5:
        return 2
    if value <= 4 and value >= 2:
        return 1
    else:
        return 0
df['target'] = df['rating'].apply(lambda x:score_to_Target(x))

TypeError: '>=' not supported between instances of 'str' and 'int'

In [20]:
rating_classes = list(df["target"])
sentiment_values  = list(df["predicted"])

from sklearn.metrics import classification_report
target_names = ["negative", "neutral", "positive"]
print(classification_report(rating_classes, sentiment_values, target_names=target_names))

              precision    recall  f1-score   support

    negative       0.78      0.04      0.08      1500
     neutral       0.36      0.93      0.52      1500
    positive       0.80      0.28      0.41      1500

    accuracy                           0.42      4500
   macro avg       0.65      0.42      0.34      4500
weighted avg       0.65      0.42      0.34      4500

